In [4]:
# ============================================
# 0. Imports & Spark session
# ============================================

import time
import builtins  # <-- IMPORTANT
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    avg,
    round as spark_round,   # Spark round ONLY for Columns
    count,
    col,
    sum as _sum
)

spark = (
    SparkSession.builder
    .appName("PostgresVsSparkBenchmark")
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.2")
    .config("spark.eventLog.enabled", "true")
    .config("spark.eventLog.dir", "/tmp/spark-events")
    .config("spark.history.fs.logDirectory", "/tmp/spark-events")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.default.parallelism", "4")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

In [5]:
# ============================================
# 1. JDBC connection config
# ============================================

jdbc_url = "jdbc:postgresql://postgres:5432/postgres"
jdbc_props = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

In [6]:
# ============================================
# 2. Load data from PostgreSQL
# ============================================

print("\n=== Loading orders_big from PostgreSQL ===")

start = time.time()

df_big = spark.read.jdbc(
    url=jdbc_url,
    table="orders_big",
    properties=jdbc_props
)

# Force materialization
row_count = df_big.count()

print(f"Rows loaded: {row_count}")
print("Load time:", builtins.round(time.time() - start, 2), "seconds")

# Register temp view
df_big.createOrReplaceTempView("orders_big")


=== Loading orders_big from PostgreSQL ===
Rows loaded: 1000000
Load time: 3.22 seconds


In [9]:
q_a = spark.sql("""
SELECT *
FROM orders_big
ORDER BY price_per_unit DESC
LIMIT 1;
""")

q_a.collect()

[Row(id=841292, order_name='Emma Brown', product_category='Automotive', quantity=3, price_per_unit=2000.0, order_date=datetime.date(2024, 10, 11), country='Italy')]

In [10]:
q_b = spark.sql("""
SELECT
  product_category,
  SUM(quantity) AS total_quantity
FROM orders_big
GROUP BY product_category
ORDER BY total_quantity DESC
LIMIT 3;
""")

q_b.collect()

[Row(product_category='Health & Beauty', total_quantity=300842),
 Row(product_category='Electronics', total_quantity=300804),
 Row(product_category='Toys', total_quantity=300598)]

In [11]:
q_c = spark.sql("""
SELECT
  product_category,
  SUM(price_per_unit * quantity) AS total_revenue
FROM orders_big
GROUP BY product_category
ORDER BY total_revenue DESC;
""")

q_c.collect()

[Row(product_category='Automotive', total_revenue=306589798.8599943),
 Row(product_category='Electronics', total_revenue=241525009.45000267),
 Row(product_category='Home & Garden', total_revenue=78023780.0900001),
 Row(product_category='Sports', total_revenue=61848990.830000326),
 Row(product_category='Health & Beauty', total_revenue=46599817.8900003),
 Row(product_category='Office Supplies', total_revenue=38276061.640000574),
 Row(product_category='Fashion', total_revenue=31566368.219999947),
 Row(product_category='Toys', total_revenue=23271039.019999716),
 Row(product_category='Grocery', total_revenue=15268355.660000028),
 Row(product_category='Books', total_revenue=12731976.03999989)]

In [12]:
q_d = spark.sql("""
SELECT
  order_name AS customer_name,
  SUM(price_per_unit * quantity) AS total_spent
FROM orders_big
GROUP BY order_name
ORDER BY total_spent DESC
LIMIT 10;
""")

q_d.collect()

[Row(customer_name='Carol Taylor', total_spent=991179.1800000003),
 Row(customer_name='Nina Lopez', total_spent=975444.9499999998),
 Row(customer_name='Daniel Jackson', total_spent=959344.4800000001),
 Row(customer_name='Carol Lewis', total_spent=947708.5700000002),
 Row(customer_name='Daniel Young', total_spent=946030.1400000004),
 Row(customer_name='Alice Martinez', total_spent=935100.0199999999),
 Row(customer_name='Ethan Perez', total_spent=934841.2399999991),
 Row(customer_name='Leo Lee', total_spent=934796.4799999993),
 Row(customer_name='Eve Young', total_spent=933176.8599999989),
 Row(customer_name='Ivy Rodriguez', total_spent=925742.6400000005)]